# FINRA Daily Short Sale Data Analysis
## MSOS & MSOX Short Volume Analysis

This notebook scrapes FINRA's daily short sale data and analyzes:
- Short volume trends for MSOS and MSOX
- Short exempt volume patterns and anomalies
- Daily/weekly/monthly averages
- Days with highest short volume
- Discrepancies and unusual patterns

## 1. Setup and Configuration

In [ ]:
# Install required packages (if needed)
!pip install pandas numpy matplotlib seaborn requests pandas_market_calendars -q

In [ ]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import pandas_market_calendars as mcal
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
import warnings
import time

warnings.filterwarnings('ignore')
plt.style.use('seaborn-v0_8-darkgrid')

print("Libraries loaded successfully!")

In [ ]:
# =============================================================================
# CONFIGURATION - Modify these values as needed
# =============================================================================

# Number of months to scrape (going back from today)
MONTHS_TO_SCRAPE = 3

# Symbols to analyze
TARGET_SYMBOLS = ['MSOS', 'MSOX']

# FINRA URL template
FINRA_URL_TEMPLATE = "https://cdn.finra.org/equity/regsho/daily/CNMSshvol{date}.txt"

# Request delay (seconds) to be respectful to the server
REQUEST_DELAY = 0.25

print(f"Configuration:")
print(f"  - Months to scrape: {MONTHS_TO_SCRAPE}")
print(f"  - Target symbols: {TARGET_SYMBOLS}")

## 2. Trading Days Calendar Generation

In [ ]:
def get_trading_days(months_back):
    """
    Get list of trading days for the specified number of months back.
    Uses NYSE calendar to identify valid trading days.
    """
    nyse = mcal.get_calendar('NYSE')
    
    end_date = datetime.now()
    start_date = end_date - relativedelta(months=months_back)
    
    # Get trading schedule
    schedule = nyse.schedule(start_date=start_date, end_date=end_date)
    trading_days = schedule.index.tolist()
    
    # Convert to date strings in YYYYMMDD format
    trading_dates = [d.strftime('%Y%m%d') for d in trading_days]
    
    print(f"Found {len(trading_dates)} trading days from {start_date.strftime('%Y-%m-%d')} to {end_date.strftime('%Y-%m-%d')}")
    
    return trading_dates

# Generate trading days list
trading_dates = get_trading_days(MONTHS_TO_SCRAPE)
print(f"\nFirst 5 dates: {trading_dates[:5]}")
print(f"Last 5 dates: {trading_dates[-5:]}")

## 3. Data Scraping Functions

In [ ]:
def fetch_finra_data(date_str, target_symbols=None):
    """
    Fetch FINRA short sale data for a specific date.
    
    Args:
        date_str: Date in YYYYMMDD format
        target_symbols: List of symbols to filter (None for all)
    
    Returns:
        DataFrame with short sale data or None if fetch fails
    """
    url = FINRA_URL_TEMPLATE.format(date=date_str)
    
    try:
        response = requests.get(url, timeout=30)
        
        if response.status_code == 200:
            # Parse the pipe-delimited data
            df = pd.read_csv(
                StringIO(response.text),
                sep='|',
                dtype={
                    'Date': str,
                    'Symbol': str,
                    'ShortVolume': 'Int64',
                    'ShortExemptVolume': 'Int64',
                    'TotalVolume': 'Int64',
                    'Market': str
                }
            )
            
            # Filter for target symbols if specified
            if target_symbols:
                df = df[df['Symbol'].isin(target_symbols)]
            
            return df
        elif response.status_code == 404:
            return None  # Data not available for this date
        else:
            print(f"  Warning: Status {response.status_code} for {date_str}")
            return None
            
    except Exception as e:
        print(f"  Error fetching {date_str}: {str(e)}")
        return None


def scrape_all_data(trading_dates, target_symbols):
    """
    Scrape FINRA data for all trading dates.
    
    Returns:
        Combined DataFrame with all data
    """
    all_data = []
    successful = 0
    failed = 0
    
    print(f"Scraping {len(trading_dates)} trading days for {target_symbols}...\n")
    
    for i, date_str in enumerate(trading_dates):
        df = fetch_finra_data(date_str, target_symbols)
        
        if df is not None and len(df) > 0:
            all_data.append(df)
            successful += 1
        else:
            failed += 1
        
        # Progress indicator
        if (i + 1) % 20 == 0 or (i + 1) == len(trading_dates):
            print(f"  Progress: {i + 1}/{len(trading_dates)} ({successful} successful, {failed} failed)")
        
        # Rate limiting
        time.sleep(REQUEST_DELAY)
    
    if all_data:
        combined_df = pd.concat(all_data, ignore_index=True)
        print(f"\nTotal records collected: {len(combined_df)}")
        return combined_df
    else:
        print("No data collected!")
        return pd.DataFrame()

In [ ]:
# Scrape all data
raw_data = scrape_all_data(trading_dates, TARGET_SYMBOLS)

if len(raw_data) > 0:
    print("\nSample data:")
    display(raw_data.head(10))

## 4. Data Preprocessing

In [ ]:
def preprocess_data(df):
    """
    Clean and preprocess the raw FINRA data.
    """
    if df.empty:
        return df
    
    df = df.copy()
    
    # Convert date to datetime
    df['Date'] = pd.to_datetime(df['Date'], format='%Y%m%d')
    
    # Calculate derived metrics
    df['ShortPercent'] = (df['ShortVolume'] / df['TotalVolume'] * 100).round(2)
    df['ShortExemptPercent'] = (df['ShortExemptVolume'] / df['TotalVolume'] * 100).round(4)
    df['ShortExemptRatio'] = (df['ShortExemptVolume'] / df['ShortVolume'] * 100).round(4)
    df['NonShortVolume'] = df['TotalVolume'] - df['ShortVolume']
    
    # Add time-based columns for grouping
    df['DayOfWeek'] = df['Date'].dt.day_name()
    df['WeekNumber'] = df['Date'].dt.isocalendar().week
    df['YearWeek'] = df['Date'].dt.strftime('%Y-W%W')
    df['Month'] = df['Date'].dt.to_period('M')
    df['YearMonth'] = df['Date'].dt.strftime('%Y-%m')
    
    # Sort by date and symbol
    df = df.sort_values(['Symbol', 'Date']).reset_index(drop=True)
    
    return df

# Preprocess the data
data = preprocess_data(raw_data)

if len(data) > 0:
    print("Preprocessed data columns:")
    print(data.columns.tolist())
    print("\nData shape:", data.shape)
    print("\nData types:")
    print(data.dtypes)
    print("\nSample preprocessed data:")
    display(data.head(10))

## 5. Summary Statistics

In [ ]:
def generate_summary_stats(df):
    """
    Generate comprehensive summary statistics for each symbol.
    """
    if df.empty:
        return None
    
    summary = df.groupby('Symbol').agg({
        'Date': ['min', 'max', 'count'],
        'ShortVolume': ['sum', 'mean', 'std', 'min', 'max', 'median'],
        'ShortExemptVolume': ['sum', 'mean', 'std', 'min', 'max', 'median'],
        'TotalVolume': ['sum', 'mean', 'std', 'min', 'max', 'median'],
        'ShortPercent': ['mean', 'std', 'min', 'max', 'median'],
        'ShortExemptPercent': ['mean', 'std', 'min', 'max', 'median'],
        'ShortExemptRatio': ['mean', 'std', 'min', 'max', 'median']
    }).round(2)
    
    # Flatten column names
    summary.columns = ['_'.join(col).strip() for col in summary.columns.values]
    
    return summary

# Generate summary
summary_stats = generate_summary_stats(data)

if summary_stats is not None:
    print("=" * 80)
    print("OVERALL SUMMARY STATISTICS")
    print("=" * 80)
    display(summary_stats.T)

In [ ]:
# Per-symbol detailed summary
for symbol in TARGET_SYMBOLS:
    symbol_data = data[data['Symbol'] == symbol]
    if len(symbol_data) > 0:
        print(f"\n{'=' * 80}")
        print(f"{symbol} - DETAILED STATISTICS")
        print(f"{'=' * 80}")
        print(f"\nDate Range: {symbol_data['Date'].min().strftime('%Y-%m-%d')} to {symbol_data['Date'].max().strftime('%Y-%m-%d')}")
        print(f"Trading Days: {len(symbol_data)}")
        print(f"\nVolume Statistics:")
        print(f"  Total Volume (sum): {symbol_data['TotalVolume'].sum():,}")
        print(f"  Total Short Volume (sum): {symbol_data['ShortVolume'].sum():,}")
        print(f"  Total Short Exempt Volume (sum): {symbol_data['ShortExemptVolume'].sum():,}")
        print(f"\nShort Percentage:")
        print(f"  Average: {symbol_data['ShortPercent'].mean():.2f}%")
        print(f"  Median: {symbol_data['ShortPercent'].median():.2f}%")
        print(f"  Std Dev: {symbol_data['ShortPercent'].std():.2f}%")
        print(f"  Range: {symbol_data['ShortPercent'].min():.2f}% - {symbol_data['ShortPercent'].max():.2f}%")
        print(f"\nShort Exempt Analysis:")
        print(f"  Avg Short Exempt %: {symbol_data['ShortExemptPercent'].mean():.4f}%")
        print(f"  Avg Short Exempt Ratio (of short volume): {symbol_data['ShortExemptRatio'].mean():.4f}%")
        print(f"  Max Short Exempt Volume: {symbol_data['ShortExemptVolume'].max():,}")
        print(f"  Days with Short Exempt > 0: {len(symbol_data[symbol_data['ShortExemptVolume'] > 0])}")

## 6. Days with Highest Short Volume

In [ ]:
def get_top_short_days(df, n=20):
    """
    Get the top N days by short volume for each symbol.
    """
    results = {}
    
    for symbol in df['Symbol'].unique():
        symbol_df = df[df['Symbol'] == symbol].copy()
        top_days = symbol_df.nlargest(n, 'ShortVolume')[[
            'Date', 'Symbol', 'ShortVolume', 'ShortExemptVolume', 
            'TotalVolume', 'ShortPercent', 'ShortExemptRatio'
        ]]
        results[symbol] = top_days
    
    return results

# Get top short volume days
top_short_days = get_top_short_days(data, n=20)

for symbol, df_top in top_short_days.items():
    print(f"\n{'=' * 80}")
    print(f"TOP 20 HIGHEST SHORT VOLUME DAYS - {symbol}")
    print(f"{'=' * 80}")
    df_display = df_top.copy()
    df_display['Date'] = df_display['Date'].dt.strftime('%Y-%m-%d')
    df_display['ShortVolume'] = df_display['ShortVolume'].apply(lambda x: f"{x:,}")
    df_display['ShortExemptVolume'] = df_display['ShortExemptVolume'].apply(lambda x: f"{x:,}")
    df_display['TotalVolume'] = df_display['TotalVolume'].apply(lambda x: f"{x:,}")
    display(df_display)

In [ ]:
# Top days by Short Exempt Volume
def get_top_exempt_days(df, n=20):
    """
    Get the top N days by short exempt volume for each symbol.
    """
    results = {}
    
    for symbol in df['Symbol'].unique():
        symbol_df = df[df['Symbol'] == symbol].copy()
        # Filter to days with exempt volume > 0
        symbol_df = symbol_df[symbol_df['ShortExemptVolume'] > 0]
        top_days = symbol_df.nlargest(n, 'ShortExemptVolume')[[
            'Date', 'Symbol', 'ShortExemptVolume', 'ShortVolume',
            'TotalVolume', 'ShortExemptPercent', 'ShortExemptRatio'
        ]]
        results[symbol] = top_days
    
    return results

top_exempt_days = get_top_exempt_days(data, n=20)

for symbol, df_top in top_exempt_days.items():
    print(f"\n{'=' * 80}")
    print(f"TOP 20 HIGHEST SHORT EXEMPT VOLUME DAYS - {symbol}")
    print(f"{'=' * 80}")
    if len(df_top) > 0:
        df_display = df_top.copy()
        df_display['Date'] = df_display['Date'].dt.strftime('%Y-%m-%d')
        df_display['ShortExemptVolume'] = df_display['ShortExemptVolume'].apply(lambda x: f"{x:,}")
        df_display['ShortVolume'] = df_display['ShortVolume'].apply(lambda x: f"{x:,}")
        df_display['TotalVolume'] = df_display['TotalVolume'].apply(lambda x: f"{x:,}")
        display(df_display)
    else:
        print("No short exempt volume recorded for this period.")

## 7. Average Short Volume by Time Period

In [ ]:
# Daily averages by day of week
def analyze_by_day_of_week(df):
    """
    Analyze short volume patterns by day of week.
    """
    day_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
    
    results = {}
    for symbol in df['Symbol'].unique():
        symbol_df = df[df['Symbol'] == symbol]
        
        daily_avg = symbol_df.groupby('DayOfWeek').agg({
            'ShortVolume': ['mean', 'sum', 'count'],
            'ShortExemptVolume': ['mean', 'sum'],
            'TotalVolume': ['mean', 'sum'],
            'ShortPercent': 'mean',
            'ShortExemptRatio': 'mean'
        }).round(2)
        
        daily_avg.columns = ['_'.join(col).strip() for col in daily_avg.columns.values]
        daily_avg = daily_avg.reindex(day_order)
        results[symbol] = daily_avg
    
    return results

dow_analysis = analyze_by_day_of_week(data)

for symbol, dow_df in dow_analysis.items():
    print(f"\n{'=' * 80}")
    print(f"AVERAGE SHORT VOLUME BY DAY OF WEEK - {symbol}")
    print(f"{'=' * 80}")
    display(dow_df)

In [ ]:
# Weekly analysis
def analyze_by_week(df):
    """
    Analyze short volume by week.
    """
    results = {}
    for symbol in df['Symbol'].unique():
        symbol_df = df[df['Symbol'] == symbol]
        
        weekly = symbol_df.groupby('YearWeek').agg({
            'Date': ['min', 'max'],
            'ShortVolume': ['sum', 'mean'],
            'ShortExemptVolume': ['sum', 'mean'],
            'TotalVolume': ['sum', 'mean'],
            'ShortPercent': 'mean',
            'ShortExemptRatio': 'mean'
        }).round(2)
        
        weekly.columns = ['_'.join(col).strip() for col in weekly.columns.values]
        weekly = weekly.sort_index()
        results[symbol] = weekly
    
    return results

weekly_analysis = analyze_by_week(data)

for symbol, weekly_df in weekly_analysis.items():
    print(f"\n{'=' * 80}")
    print(f"WEEKLY SHORT VOLUME ANALYSIS - {symbol}")
    print(f"{'=' * 80}")
    display(weekly_df)

In [ ]:
# Monthly analysis
def analyze_by_month(df):
    """
    Analyze short volume by month.
    """
    results = {}
    for symbol in df['Symbol'].unique():
        symbol_df = df[df['Symbol'] == symbol]
        
        monthly = symbol_df.groupby('YearMonth').agg({
            'Date': 'count',
            'ShortVolume': ['sum', 'mean', 'std'],
            'ShortExemptVolume': ['sum', 'mean'],
            'TotalVolume': ['sum', 'mean'],
            'ShortPercent': ['mean', 'std'],
            'ShortExemptRatio': ['mean', 'max']
        }).round(2)
        
        monthly.columns = ['_'.join(col).strip() for col in monthly.columns.values]
        monthly = monthly.rename(columns={'Date_count': 'TradingDays'})
        monthly = monthly.sort_index()
        results[symbol] = monthly
    
    return results

monthly_analysis = analyze_by_month(data)

for symbol, monthly_df in monthly_analysis.items():
    print(f"\n{'=' * 80}")
    print(f"MONTHLY SHORT VOLUME ANALYSIS - {symbol}")
    print(f"{'=' * 80}")
    display(monthly_df)

## 8. Short Exempt Analysis & Anomaly Detection

In [ ]:
def analyze_short_exempt(df):
    """
    Comprehensive analysis of short exempt volume patterns and anomalies.
    """
    print("=" * 80)
    print("SHORT EXEMPT VOLUME ANALYSIS")
    print("=" * 80)
    
    for symbol in df['Symbol'].unique():
        symbol_df = df[df['Symbol'] == symbol].copy()
        
        print(f"\n{'-' * 60}")
        print(f"{symbol}")
        print(f"{'-' * 60}")
        
        # Basic stats
        total_exempt = symbol_df['ShortExemptVolume'].sum()
        total_short = symbol_df['ShortVolume'].sum()
        days_with_exempt = len(symbol_df[symbol_df['ShortExemptVolume'] > 0])
        total_days = len(symbol_df)
        
        print(f"\nOverview:")
        print(f"  Total Short Exempt Volume: {total_exempt:,}")
        print(f"  Total Short Volume: {total_short:,}")
        print(f"  Overall Exempt Ratio: {(total_exempt/total_short*100):.4f}%")
        print(f"  Days with Exempt Volume: {days_with_exempt}/{total_days} ({days_with_exempt/total_days*100:.1f}%)")
        
        # Statistical analysis for anomaly detection
        exempt_data = symbol_df[symbol_df['ShortExemptVolume'] > 0]['ShortExemptVolume']
        
        if len(exempt_data) > 0:
            mean_exempt = exempt_data.mean()
            std_exempt = exempt_data.std()
            median_exempt = exempt_data.median()
            
            print(f"\nExempt Volume Statistics (days with exempt > 0):")
            print(f"  Mean: {mean_exempt:,.0f}")
            print(f"  Median: {median_exempt:,.0f}")
            print(f"  Std Dev: {std_exempt:,.0f}")
            print(f"  Min: {exempt_data.min():,}")
            print(f"  Max: {exempt_data.max():,}")
            
            # Identify anomalies (> 2 std deviations)
            if std_exempt > 0:
                threshold_2std = mean_exempt + (2 * std_exempt)
                threshold_3std = mean_exempt + (3 * std_exempt)
                
                anomalies_2std = symbol_df[symbol_df['ShortExemptVolume'] > threshold_2std]
                anomalies_3std = symbol_df[symbol_df['ShortExemptVolume'] > threshold_3std]
                
                print(f"\nAnomaly Detection:")
                print(f"  2 Std Dev Threshold: {threshold_2std:,.0f}")
                print(f"  Days exceeding 2 std: {len(anomalies_2std)}")
                print(f"  3 Std Dev Threshold: {threshold_3std:,.0f}")
                print(f"  Days exceeding 3 std: {len(anomalies_3std)}")
                
                if len(anomalies_2std) > 0:
                    print(f"\n  Anomalous Days (>2 std dev):")
                    for _, row in anomalies_2std.iterrows():
                        print(f"    {row['Date'].strftime('%Y-%m-%d')}: Exempt={row['ShortExemptVolume']:,} "
                              f"({row['ShortExemptRatio']:.2f}% of short vol)")
        
        # Short Exempt Ratio analysis
        ratio_data = symbol_df[symbol_df['ShortExemptRatio'] > 0]['ShortExemptRatio']
        if len(ratio_data) > 0:
            print(f"\nShort Exempt Ratio Analysis:")
            print(f"  Mean Ratio: {ratio_data.mean():.4f}%")
            print(f"  Max Ratio: {ratio_data.max():.4f}%")
            
            # Days with unusually high ratio
            high_ratio_threshold = ratio_data.mean() + (2 * ratio_data.std())
            high_ratio_days = symbol_df[symbol_df['ShortExemptRatio'] > high_ratio_threshold]
            
            if len(high_ratio_days) > 0:
                print(f"\n  Days with High Exempt Ratio (>{high_ratio_threshold:.2f}%):")
                for _, row in high_ratio_days.head(10).iterrows():
                    print(f"    {row['Date'].strftime('%Y-%m-%d')}: {row['ShortExemptRatio']:.4f}% "
                          f"(Exempt: {row['ShortExemptVolume']:,})")

analyze_short_exempt(data)

## 9. Discrepancy Analysis

In [ ]:
def analyze_discrepancies(df):
    """
    Analyze potential discrepancies and unusual patterns in the data.
    """
    print("=" * 80)
    print("DISCREPANCY ANALYSIS")
    print("=" * 80)
    
    for symbol in df['Symbol'].unique():
        symbol_df = df[df['Symbol'] == symbol].copy().sort_values('Date')
        
        print(f"\n{'-' * 60}")
        print(f"{symbol}")
        print(f"{'-' * 60}")
        
        # 1. Days where short volume > total volume (should never happen)
        invalid_short = symbol_df[symbol_df['ShortVolume'] > symbol_df['TotalVolume']]
        print(f"\n1. Data Integrity Check:")
        print(f"   Days with ShortVolume > TotalVolume: {len(invalid_short)}")
        if len(invalid_short) > 0:
            print("   WARNING: Data integrity issue detected!")
            display(invalid_short[['Date', 'ShortVolume', 'TotalVolume']])
        
        # 2. Days where short + exempt doesn't make sense
        symbol_df['ImpliedNonExemptShort'] = symbol_df['ShortVolume'] - symbol_df['ShortExemptVolume']
        invalid_exempt = symbol_df[symbol_df['ImpliedNonExemptShort'] < 0]
        print(f"\n2. Short Exempt Consistency:")
        print(f"   Days with ShortExempt > ShortVolume: {len(invalid_exempt)}")
        if len(invalid_exempt) > 0:
            print("   WARNING: Short exempt exceeds total short volume!")
            display(invalid_exempt[['Date', 'ShortVolume', 'ShortExemptVolume']])
        
        # 3. Unusual short percentage spikes (>2 std dev from mean)
        mean_pct = symbol_df['ShortPercent'].mean()
        std_pct = symbol_df['ShortPercent'].std()
        
        high_short_pct = symbol_df[symbol_df['ShortPercent'] > mean_pct + (2 * std_pct)]
        low_short_pct = symbol_df[symbol_df['ShortPercent'] < mean_pct - (2 * std_pct)]
        
        print(f"\n3. Short Percentage Outliers:")
        print(f"   Mean Short %: {mean_pct:.2f}%")
        print(f"   Std Dev: {std_pct:.2f}%")
        print(f"   Days with unusually HIGH short % (>{mean_pct + (2 * std_pct):.2f}%): {len(high_short_pct)}")
        print(f"   Days with unusually LOW short % (<{mean_pct - (2 * std_pct):.2f}%): {len(low_short_pct)}")
        
        if len(high_short_pct) > 0:
            print(f"\n   High Short % Days:")
            for _, row in high_short_pct.head(10).iterrows():
                print(f"     {row['Date'].strftime('%Y-%m-%d')}: {row['ShortPercent']:.2f}% "
                      f"(Short: {row['ShortVolume']:,}, Total: {row['TotalVolume']:,})")
        
        # 4. Day-over-day volume changes
        symbol_df['ShortVolumePctChange'] = symbol_df['ShortVolume'].pct_change() * 100
        symbol_df['TotalVolumePctChange'] = symbol_df['TotalVolume'].pct_change() * 100
        
        large_changes = symbol_df[abs(symbol_df['ShortVolumePctChange']) > 200].dropna()
        
        print(f"\n4. Large Day-over-Day Changes (>200%):")
        print(f"   Days with large short volume swings: {len(large_changes)}")
        
        if len(large_changes) > 0:
            print(f"\n   Significant Volume Changes:")
            for _, row in large_changes.head(10).iterrows():
                print(f"     {row['Date'].strftime('%Y-%m-%d')}: {row['ShortVolumePctChange']:+.1f}% change "
                      f"(Short: {row['ShortVolume']:,})")
        
        # 5. Exempt volume correlation with total volume
        if symbol_df['ShortExemptVolume'].sum() > 0:
            exempt_corr = symbol_df['ShortExemptVolume'].corr(symbol_df['TotalVolume'])
            exempt_short_corr = symbol_df['ShortExemptVolume'].corr(symbol_df['ShortVolume'])
            
            print(f"\n5. Short Exempt Correlation Analysis:")
            print(f"   Exempt vs Total Volume correlation: {exempt_corr:.4f}")
            print(f"   Exempt vs Short Volume correlation: {exempt_short_corr:.4f}")
            
            if exempt_corr < 0.3:
                print("   NOTE: Weak correlation between exempt and total volume - "
                      "exempt activity may not follow overall market patterns")

analyze_discrepancies(data)

## 10. Visualization

In [ ]:
def create_visualizations(df):
    """
    Create comprehensive visualizations for short sale analysis.
    """
    for symbol in df['Symbol'].unique():
        symbol_df = df[df['Symbol'] == symbol].sort_values('Date')
        
        fig, axes = plt.subplots(4, 2, figsize=(16, 20))
        fig.suptitle(f'{symbol} - Short Sale Analysis Dashboard', fontsize=16, fontweight='bold')
        
        # 1. Short Volume Over Time
        ax1 = axes[0, 0]
        ax1.bar(symbol_df['Date'], symbol_df['ShortVolume'], alpha=0.7, label='Short Volume', color='red')
        ax1.set_title('Daily Short Volume')
        ax1.set_xlabel('Date')
        ax1.set_ylabel('Volume')
        ax1.tick_params(axis='x', rotation=45)
        ax1.legend()
        
        # 2. Short Percentage Over Time
        ax2 = axes[0, 1]
        ax2.plot(symbol_df['Date'], symbol_df['ShortPercent'], color='darkred', linewidth=1)
        ax2.axhline(y=symbol_df['ShortPercent'].mean(), color='blue', linestyle='--', label=f"Mean: {symbol_df['ShortPercent'].mean():.1f}%")
        ax2.fill_between(symbol_df['Date'], symbol_df['ShortPercent'], alpha=0.3, color='red')
        ax2.set_title('Short Volume as % of Total Volume')
        ax2.set_xlabel('Date')
        ax2.set_ylabel('Short %')
        ax2.tick_params(axis='x', rotation=45)
        ax2.legend()
        
        # 3. Short Exempt Volume Over Time
        ax3 = axes[1, 0]
        ax3.bar(symbol_df['Date'], symbol_df['ShortExemptVolume'], alpha=0.7, color='orange', label='Short Exempt')
        ax3.set_title('Daily Short Exempt Volume')
        ax3.set_xlabel('Date')
        ax3.set_ylabel('Volume')
        ax3.tick_params(axis='x', rotation=45)
        ax3.legend()
        
        # 4. Short Exempt Ratio Over Time
        ax4 = axes[1, 1]
        ax4.plot(symbol_df['Date'], symbol_df['ShortExemptRatio'], color='darkorange', linewidth=1)
        ax4.set_title('Short Exempt as % of Short Volume')
        ax4.set_xlabel('Date')
        ax4.set_ylabel('Exempt Ratio %')
        ax4.tick_params(axis='x', rotation=45)
        
        # 5. Volume Comparison (Stacked)
        ax5 = axes[2, 0]
        ax5.bar(symbol_df['Date'], symbol_df['NonShortVolume'], label='Non-Short', alpha=0.7, color='green')
        ax5.bar(symbol_df['Date'], symbol_df['ShortVolume'], bottom=symbol_df['NonShortVolume'], 
                label='Short', alpha=0.7, color='red')
        ax5.set_title('Total Volume Breakdown')
        ax5.set_xlabel('Date')
        ax5.set_ylabel('Volume')
        ax5.tick_params(axis='x', rotation=45)
        ax5.legend()
        
        # 6. Short % Distribution
        ax6 = axes[2, 1]
        ax6.hist(symbol_df['ShortPercent'], bins=30, edgecolor='black', alpha=0.7, color='red')
        ax6.axvline(symbol_df['ShortPercent'].mean(), color='blue', linestyle='--', 
                    label=f"Mean: {symbol_df['ShortPercent'].mean():.1f}%")
        ax6.axvline(symbol_df['ShortPercent'].median(), color='green', linestyle='--',
                    label=f"Median: {symbol_df['ShortPercent'].median():.1f}%")
        ax6.set_title('Distribution of Short %')
        ax6.set_xlabel('Short %')
        ax6.set_ylabel('Frequency')
        ax6.legend()
        
        # 7. Day of Week Analysis
        ax7 = axes[3, 0]
        dow_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday']
        dow_avg = symbol_df.groupby('DayOfWeek')['ShortVolume'].mean().reindex(dow_order)
        ax7.bar(dow_avg.index, dow_avg.values, color='purple', alpha=0.7)
        ax7.set_title('Average Short Volume by Day of Week')
        ax7.set_xlabel('Day')
        ax7.set_ylabel('Avg Short Volume')
        ax7.tick_params(axis='x', rotation=45)
        
        # 8. Monthly Trend
        ax8 = axes[3, 1]
        monthly = symbol_df.groupby('YearMonth').agg({
            'ShortVolume': 'sum',
            'ShortExemptVolume': 'sum'
        })
        x_pos = range(len(monthly))
        width = 0.35
        ax8.bar([p - width/2 for p in x_pos], monthly['ShortVolume'], width, label='Short Vol', color='red', alpha=0.7)
        ax8.bar([p + width/2 for p in x_pos], monthly['ShortExemptVolume'], width, label='Exempt Vol', color='orange', alpha=0.7)
        ax8.set_title('Monthly Short Volume Comparison')
        ax8.set_xlabel('Month')
        ax8.set_ylabel('Volume')
        ax8.set_xticks(x_pos)
        ax8.set_xticklabels(monthly.index, rotation=45)
        ax8.legend()
        
        plt.tight_layout()
        plt.subplots_adjust(top=0.95)
        plt.show()

create_visualizations(data)

In [ ]:
# Comparison chart for MSOS vs MSOX
if len(TARGET_SYMBOLS) > 1 and all(s in data['Symbol'].unique() for s in TARGET_SYMBOLS):
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    fig.suptitle('MSOS vs MSOX Comparison', fontsize=16, fontweight='bold')
    
    colors = {'MSOS': 'blue', 'MSOX': 'red'}
    
    # 1. Short % Comparison
    ax1 = axes[0, 0]
    for symbol in TARGET_SYMBOLS:
        symbol_df = data[data['Symbol'] == symbol].sort_values('Date')
        ax1.plot(symbol_df['Date'], symbol_df['ShortPercent'], label=symbol, color=colors.get(symbol, 'gray'))
    ax1.set_title('Short % Comparison Over Time')
    ax1.set_xlabel('Date')
    ax1.set_ylabel('Short %')
    ax1.legend()
    ax1.tick_params(axis='x', rotation=45)
    
    # 2. Short Exempt Ratio Comparison
    ax2 = axes[0, 1]
    for symbol in TARGET_SYMBOLS:
        symbol_df = data[data['Symbol'] == symbol].sort_values('Date')
        ax2.plot(symbol_df['Date'], symbol_df['ShortExemptRatio'], label=symbol, color=colors.get(symbol, 'gray'))
    ax2.set_title('Short Exempt Ratio Comparison')
    ax2.set_xlabel('Date')
    ax2.set_ylabel('Exempt Ratio %')
    ax2.legend()
    ax2.tick_params(axis='x', rotation=45)
    
    # 3. Total Short Volume Comparison
    ax3 = axes[1, 0]
    monthly_comparison = data.groupby(['YearMonth', 'Symbol'])['ShortVolume'].sum().unstack()
    monthly_comparison.plot(kind='bar', ax=ax3, color=[colors.get(s, 'gray') for s in monthly_comparison.columns])
    ax3.set_title('Monthly Short Volume by Symbol')
    ax3.set_xlabel('Month')
    ax3.set_ylabel('Total Short Volume')
    ax3.tick_params(axis='x', rotation=45)
    ax3.legend(title='Symbol')
    
    # 4. Average Short % by Symbol
    ax4 = axes[1, 1]
    avg_short_pct = data.groupby('Symbol')['ShortPercent'].agg(['mean', 'std'])
    ax4.bar(avg_short_pct.index, avg_short_pct['mean'], 
            yerr=avg_short_pct['std'], capsize=5,
            color=[colors.get(s, 'gray') for s in avg_short_pct.index], alpha=0.7)
    ax4.set_title('Average Short % by Symbol (with Std Dev)')
    ax4.set_xlabel('Symbol')
    ax4.set_ylabel('Average Short %')
    
    plt.tight_layout()
    plt.subplots_adjust(top=0.93)
    plt.show()

## 11. Export Data

In [ ]:
# Export all data to CSV files
from google.colab import files

# Main data export
export_df = data.copy()
export_df['Date'] = export_df['Date'].dt.strftime('%Y-%m-%d')
export_df['Month'] = export_df['Month'].astype(str)

filename = f"finra_short_data_{TARGET_SYMBOLS[0]}_{MONTHS_TO_SCRAPE}mo.csv"
export_df.to_csv(filename, index=False)
print(f"Main data exported to: {filename}")

# Download the file
files.download(filename)

In [ ]:
# Export summary reports
summary_filename = f"finra_summary_{TARGET_SYMBOLS[0]}_{MONTHS_TO_SCRAPE}mo.csv"

# Combine summaries
with open(summary_filename, 'w') as f:
    f.write("FINRA SHORT SALE ANALYSIS SUMMARY\n")
    f.write(f"Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n")
    f.write(f"Symbols: {', '.join(TARGET_SYMBOLS)}\n")
    f.write(f"Period: {MONTHS_TO_SCRAPE} months\n\n")
    
    for symbol in TARGET_SYMBOLS:
        symbol_df = data[data['Symbol'] == symbol]
        if len(symbol_df) > 0:
            f.write(f"\n{'='*60}\n")
            f.write(f"{symbol} SUMMARY\n")
            f.write(f"{'='*60}\n")
            f.write(f"Trading Days: {len(symbol_df)}\n")
            f.write(f"Date Range: {symbol_df['Date'].min()} to {symbol_df['Date'].max()}\n")
            f.write(f"Total Short Volume: {symbol_df['ShortVolume'].sum():,}\n")
            f.write(f"Total Short Exempt: {symbol_df['ShortExemptVolume'].sum():,}\n")
            f.write(f"Avg Short %: {symbol_df['ShortPercent'].mean():.2f}%\n")
            f.write(f"Avg Exempt Ratio: {symbol_df['ShortExemptRatio'].mean():.4f}%\n")

print(f"Summary exported to: {summary_filename}")
files.download(summary_filename)

## 12. Quick Reference - Key Findings

In [ ]:
# Generate a quick reference summary
print("=" * 80)
print("KEY FINDINGS SUMMARY")
print("=" * 80)

for symbol in TARGET_SYMBOLS:
    symbol_df = data[data['Symbol'] == symbol]
    if len(symbol_df) == 0:
        print(f"\n{symbol}: No data available")
        continue
        
    print(f"\n{'-' * 60}")
    print(f"{symbol}")
    print(f"{'-' * 60}")
    
    # Highest short day
    max_short_day = symbol_df.loc[symbol_df['ShortVolume'].idxmax()]
    print(f"\n📊 Highest Short Volume Day:")
    print(f"   Date: {max_short_day['Date'].strftime('%Y-%m-%d')}")
    print(f"   Short Volume: {max_short_day['ShortVolume']:,}")
    print(f"   Short %: {max_short_day['ShortPercent']:.2f}%")
    
    # Highest exempt day
    if symbol_df['ShortExemptVolume'].sum() > 0:
        max_exempt_day = symbol_df.loc[symbol_df['ShortExemptVolume'].idxmax()]
        print(f"\n⚠️ Highest Short Exempt Day:")
        print(f"   Date: {max_exempt_day['Date'].strftime('%Y-%m-%d')}")
        print(f"   Exempt Volume: {max_exempt_day['ShortExemptVolume']:,}")
        print(f"   Exempt Ratio: {max_exempt_day['ShortExemptRatio']:.4f}%")
    
    # Average metrics
    print(f"\n📈 Averages:")
    print(f"   Daily Short Volume: {symbol_df['ShortVolume'].mean():,.0f}")
    print(f"   Daily Short %: {symbol_df['ShortPercent'].mean():.2f}%")
    print(f"   Daily Exempt Volume: {symbol_df['ShortExemptVolume'].mean():,.0f}")
    
    # Trend (comparing first half to second half)
    midpoint = len(symbol_df) // 2
    first_half_avg = symbol_df.iloc[:midpoint]['ShortPercent'].mean()
    second_half_avg = symbol_df.iloc[midpoint:]['ShortPercent'].mean()
    trend = "↗️ INCREASING" if second_half_avg > first_half_avg else "↘️ DECREASING"
    print(f"\n📉 Short % Trend: {trend}")
    print(f"   First Half Avg: {first_half_avg:.2f}%")
    print(f"   Second Half Avg: {second_half_avg:.2f}%")
    
print("\n" + "=" * 80)